In [1]:
# imporiting all the requirements
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# path of the files
PATH = r'dataset/'

In [2]:
# read the train and test files
df1 = pd.read_excel(PATH + 'Data_Train.xlsx')
df2 = pd.read_excel(PATH + 'Test_set.xlsx')
print(df1.shape)
df2.shape

(10683, 11)


(2671, 10)

In [3]:
# now, combine both the dataset to perform preprocessing
final_df = pd.concat((df1, df2))
final_df.shape

(13354, 11)

In [4]:
# information about all the features and their types
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


In [5]:
# about missing values
final_df.isnull().sum()

Airline               0
Date_of_Journey       0
Source                0
Destination           0
Route                 1
Dep_Time              0
Arrival_Time          0
Duration              0
Total_Stops           1
Additional_Info       0
Price              2671
dtype: int64

## now, convert all the categorical features into numerical features

In [6]:
final_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


### convert "Date_of_Journey" into numerical feature

In [7]:
type(final_df['Date_of_Journey'])

pandas.core.series.Series

In [8]:
final_df['Date_of_Journey']

0       24/03/2019
1        1/05/2019
2        9/06/2019
3       12/05/2019
4       01/03/2019
           ...    
2666     6/06/2019
2667    27/03/2019
2668     6/03/2019
2669     6/03/2019
2670    15/06/2019
Name: Date_of_Journey, Length: 13354, dtype: object

In [9]:
# adding day, month and year columns seperately
try:
    final_df['Journey_Day'] = final_df['Date_of_Journey'].str.split('/').str[0].astype(int)  # lambda can be used.
    final_df['Journey_Month'] = final_df['Date_of_Journey'].apply(lambda val: val.split('/')[1]).astype(int)
    final_df['Journey_Year'] = final_df['Date_of_Journey'].apply(lambda val: val.split('/')[2]).astype(int)
except:
    "columns ('Date_of_Journey') doesn't exist because it is dropped by the programmer (if the row executed again)"

# now, drop the "Date_of_Journey" column
final_df.drop(columns=['Date_of_Journey'], axis=1, inplace=True, errors='ignore')
final_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


### convert "Arrival_Time, Dep_Time, Duration" into numerical feature

In [10]:
# here we have already saved the date. Now, we need to extract the time
type(final_df['Arrival_Time'].str.split(' ').str[0])

pandas.core.series.Series

In [11]:
final_df['Arrival_Time'].str.split(' ').str[0].str.split(':')

0       [01, 10]
1       [13, 15]
2       [04, 25]
3       [23, 30]
4       [21, 35]
          ...   
2666    [20, 25]
2667    [16, 55]
2668    [04, 25]
2669    [19, 15]
2670    [19, 15]
Name: Arrival_Time, Length: 13354, dtype: object

In [12]:
# now, add this data into dataset
try:
    final_df['Arrival_Hours'] = final_df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[0].astype(int)
    final_df['Arrival_Mins'] = final_df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[1].astype(int)
except:
    "columns already dropped"

# now, remove the arrival time column from the dataset
final_df.drop(columns=['Arrival_Time'], axis=1, inplace=True, errors='ignore')
final_df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year,Arrival_Hours,Arrival_Mins
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15


In [13]:
# also, remove the Dep_Time time after handling
final_df['Dep_Hours'] = final_df['Dep_Time'].str.split(':').str[0].astype(int)
final_df['Dep_Mins'] = final_df['Dep_Time'].str.split(':').str[1].astype(int)

final_df.drop('Dep_Time', axis=1, inplace=True, errors='ignore')

In [14]:
final_df.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year,Arrival_Hours,Arrival_Mins,Dep_Hours,Dep_Mins
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50


In [15]:
# # now, handle the duration and convert it into mintues only
# final_df['Duration'].str.split(' ').str[0].str.replace('h', '').astype(int)

# "ValueError: invalid literal for int() with base 10: '5m'" Handling the error using mask
"""seems in appropriate because the travel duration from mumbai to hyderbad is only 5 min.
So, we are dropping these rows or we can also change it with some other appropriate values.
"""
final_df[final_df['Duration'] == '5m']

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year,Arrival_Hours,Arrival_Mins,Dep_Hours,Dep_Mins
6474,Air India,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,5m,2 stops,No info,17327.0,6,3,2019,16,55,16,50
2660,Air India,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,5m,2 stops,No info,NaN,12,3,2019,16,55,16,50


In [16]:
mask = (final_df['Source'] == 'Mumbai') & (final_df['Destination'] == 'Hyderabad') & (final_df['Total_Stops'] == '2 stops')
five_min_appro_val = final_df[mask]
five_min_appro_val.head(1)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year,Arrival_Hours,Arrival_Mins,Dep_Hours,Dep_Mins
597,Air India,Mumbai,Hyderabad,BOM → JDH → DEL → HYD,29h 35m,2 stops,No info,25139.0,6,3,2019,15,15,9,40


In [17]:
# there are 29 time slots for duration let's pick the most appropriate here
most_appro = five_min_appro_val['Duration'].mode()  # occuring 3 times, more than others
most_appro

0    9h 20m
Name: Duration, dtype: object

In [18]:
# let's fill this value (map method is not used here because it will fill other with null)

# final_df.at[6474, 'Duration'] = most_appro[0]
# final_df.at[2660, 'Duration'] = most_appro[0]

final_df.loc[[6474, 2660], 'Duration'] = most_appro[0]

In [19]:
# now, handle the duration and convert hours into mintues and add minutes into it.
final_df['Travel_duration'] = final_df['Duration'].str.split(' ').str[0].str.replace('h', '').astype(int) * 60
final_df['duration_min'] = final_df['Duration'].str.split(' ').str[1].str.replace('m', '')

In [20]:
# now, remove the null values and fill with 0 in mins
final_df['duration_min'].fillna(value=0, inplace=True)
final_df.head(3)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year,Arrival_Hours,Arrival_Mins,Dep_Hours,Dep_Mins,Travel_duration,duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20,120,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50,420,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882.0,9,6,2019,4,25,9,25,1140,0


In [21]:
# now, merge the duration min into travel duration (add them)
final_df['Travel_duration'] += final_df['duration_min'].astype(int)
final_df.head(3)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year,Arrival_Hours,Arrival_Mins,Dep_Hours,Dep_Mins,Travel_duration,duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20,170,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50,445,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882.0,9,6,2019,4,25,9,25,1140,0


In [22]:
# now, drop the duration min and duration
final_df.drop(columns=['duration_min', 'Duration'], axis=1, inplace=True, errors='ignore')
final_df.head(3)

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year,Arrival_Hours,Arrival_Mins,Dep_Hours,Dep_Mins,Travel_duration
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897.0,24,3,2019,1,10,22,20,170
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662.0,1,5,2019,13,15,5,50,445
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882.0,9,6,2019,4,25,9,25,1140


In [25]:
# handle the categorical feature Total stops
final_df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [45]:
# There are null values there
# let's just drop this column
final_df[final_df['Total_Stops'].isnull()]
final_df.drop(index=9039, axis=0, inplace=True, errors='ignore')

In [49]:
# convert it into numerical feature
final_df['Total_Stops'] = final_df['Total_Stops'].map({'non-stop': 0, '2 stops': 2, '1 stop': 1, '3 stops': 3, '4 stops': 4})

In [50]:
final_df.head(4)

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Year,Arrival_Hours,Arrival_Mins,Dep_Hours,Dep_Mins,Travel_duration
0,IndiGo,Banglore,New Delhi,BLR → DEL,0,No info,3897.0,24,3,2019,1,10,22,20,170
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2,No info,7662.0,1,5,2019,13,15,5,50,445
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2,No info,13882.0,9,6,2019,4,25,9,25,1140
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1,No info,6218.0,12,5,2019,23,30,18,5,325
